# Auto-generate BTE annotations

In [2]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pathlib
import yaml
import pandas

The KPs have been responsive to sharing what subject-type / predicate / object-type combos have matching records in their data (aka what actually exists). 

In that case, code to automatically query the KP (underscore exists underscore) and find these combos might not be necessary.

In [3]:
## record format where each dict is a "row"
combos = [
    {  ## CHEBI-HP
        "subject-semantic": "SmallMolecule", 
        "subject-id": "CHEBI", 
        "predicate": "treats",
        "object-semantic": "PhenotypicFeature",
        "object-id": "HP"
    },
    {  ## DRUGBANK-HP
        "subject-semantic": "SmallMolecule", 
        "subject-id": "DRUGBANK", 
        "predicate": "treats",
        "object-semantic": "PhenotypicFeature",
        "object-id": "HP"
    },
    {  ## CHEBI-MONDO
        "subject-semantic": "SmallMolecule", 
        "subject-id": "CHEBI", 
        "predicate": "treats",
        "object-semantic": "Disease",
        "object-id": "MONDO"
    },
    {  ## DRUGBANK-MONDO
        "subject-semantic": "SmallMolecule", 
        "subject-id": "DRUGBANK", 
        "predicate": "treats",
        "object-semantic": "Disease",
        "object-id": "MONDO"
    },
    {  ## CHEBI-UniProtKBgene-positive
        "subject-semantic": "SmallMolecule", 
        "subject-id": "CHEBI", 
        "predicate": "entity_positively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## DRUGBANK-UniProtKBgene-positive
        "subject-semantic": "SmallMolecule", 
        "subject-id": "DRUGBANK", 
        "predicate": "entity_positively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## CHEBI-UniProtKBgene-negative
        "subject-semantic": "SmallMolecule", 
        "subject-id": "CHEBI", 
        "predicate": "entity_negatively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## DRUGBANK-UniProtKBgene-negative
        "subject-semantic": "SmallMolecule", 
        "subject-id": "DRUGBANK", 
        "predicate": "entity_negatively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## UniProtKBgene-UniProtKBgene-positive
        "subject-semantic": "Gene", 
        "subject-id": "UniProtKB", 
        "predicate": "entity_positively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## UniProtKBgene-UniProtKBgene-negative
        "subject-semantic": "Gene", 
        "subject-id": "UniProtKB", 
        "predicate": "entity_negatively_regulates_entity",
        "object-semantic": "Gene",
        "object-id": "UniProtKB"
    },
    {  ## UniProtKBgene-MONDO-1 (two underlying operations?)
        "subject-semantic": "Gene", 
        "subject-id": "UniProtKB", 
        "predicate": "contributes_to",
        "object-semantic": "Disease",
        "object-id": "MONDO"
    },
    {  ## UniProtKBgene-HP-1 (two underlying operations?)
        "subject-semantic": "Gene", 
        "subject-id": "UniProtKB", 
        "predicate": "contributes_to",
        "object-semantic": "PhenotypicFeature",
        "object-id": "HP"
    }
]

combos_df = pandas.DataFrame.from_records(combos)

In [4]:
combos_df
## may have to add columns if what we query with =/= what is in the data/records...

,subject-semantic,subject-id,predicate,object-semantic,object-id
0,SmallMolecule,CHEBI,treats,PhenotypicFeature,HP
1,SmallMolecule,DRUGBANK,treats,PhenotypicFeature,HP
2,SmallMolecule,CHEBI,treats,Disease,MONDO
3,SmallMolecule,DRUGBANK,treats,Disease,MONDO
4,SmallMolecule,CHEBI,entity_positively_regulates_entity,Gene,UniProtKB
5,SmallMolecule,DRUGBANK,entity_positively_regulates_entity,Gene,UniProtKB
6,SmallMolecule,CHEBI,entity_negatively_regulates_entity,Gene,UniProtKB
7,SmallMolecule,DRUGBANK,entity_negatively_regulates_entity,Gene,UniProtKB
8,Gene,UniProtKB,entity_positively_regulates_entity,Gene,UniProtKB
9,Gene,UniProtKB,entity_negatively_regulates_entity,Gene,UniProtKB


next is...being able to generate the operations from this...

First I have to check if the templating works with the special POST way of querying BioThings APIs...

In [ ]:
def generate_individual_x_bte_operation(input_type, output_type):
    return {
        input_type + '-' + output_type: [
            {
                "inputs": [
                    {
                        "id": TYPE_ID_MAPPING[input_type],
                        "semantic": input_type
                    }
                ],
                "outputs": [
                    {
                        "id": TYPE_ID_MAPPING[output_type],
                        "semantic": output_type
                    }
                ],
                "parameters": {
                    "q": "subject." + TYPE_ID_MAPPING[input_type] + ':"{inputs[0]}"' + " AND object.type:" + output_type,
                    "fields": "object,association",
                    "size": 1000
                },
                "predicate": "related_to",
                "response_mapping": {
                    "$ref": '#/components/x-bte-response-mapping/' + output_type
                },
                "source": "Text Mining KP",
                "supportBatch": False
            }
        ]
    }

how to generate the rev operations?

In [ ]:
## predicates and their inverse for the reverse operation?
predinverses = {
    "treats": "treated_by",
    "entity_positively_regulates_entity": "entity_positively_regulated_by_entity",
    "entity_negatively_regulates_entity": "entity_negatively_regulated_by_entity",
    "contributes_to": "contribution_from"
}